In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
sessions_df = pd.read_json('data/data_v2/sessions.jsonl')

In [ ]:
sessions_df.dropna(inplace=True)

In [ ]:
tracks_df = pd.read_json('data/data_v2/tracks.jsonl')
tracks_df

In [ ]:
tracks_df.shape

In [ ]:
columns = ["duration_ms",	
           "danceability",
           "energy",	
           "key",	
           "loudness",
           "speechiness",
           "acousticness",
           "instrumentalness",
           "liveness",	
           "valence",
           "tempo"]

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 1)
knn.fit(tracks_df[columns], tracks_df["id"])

In [ ]:
counter = 0
n_iters = 0
for session_id,row in sessions_df.groupby("session_id").first().iterrows():
    track = tracks_df[tracks_df['id'] == row['track_id']]
    x = track[columns]
    preds = knn.kneighbors(x, n_neighbors=20, return_distance=False)
    preds = tracks_df.iloc[preds[0]]["id"].to_list()
    session_tracks = sessions_df[sessions_df['session_id'] == session_id].iloc[1:]['track_id']
    session_tracks = session_tracks[session_tracks.notnull()]
    n_iters += len(session_tracks)
    for track in session_tracks:
        if track in preds:
            counter += 1

print(f"Accuracy {counter / n_iters * 100 : .2f}%")

In [ ]:
len(knn.classes_)